In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import joblib

app = Flask(__name__)


with open("match_predictor.pkl", "rb") as model_file:
    model = joblib.load(model_file)

team_stats = pd.read_csv("premier_league_2023_24.csv")  # Team statistics
head_to_head = pd.read_csv("head_to_head_stats.csv")  # Head-to-head stats

def get_team_stats(team_name):
    row = team_stats[team_stats["team"] == team_name]
    return row.iloc[0] if not row.empty else None

def get_head_to_head(home_team, away_team):
    row = head_to_head[(head_to_head["team"] == home_team) & (head_to_head["opponent"] == away_team)]
    return row.iloc[0] if not row.empty else {"head_to_head": 0, "head_to_head_goals": 0}

# API endpoint for match prediction
@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    home_team = data.get("home_team")
    away_team = data.get("away_team")

    home_stats = get_team_stats(home_team)
    away_stats = get_team_stats(away_team)
    h2h_stats = get_head_to_head(home_team, away_team)

    if home_stats is None or away_stats is None:
        return jsonify({"error": "Invalid team name"}), 400

    # Prepare input features
    features = [
        home_stats["recent_form"], home_stats["avg_goals"], home_stats["avg_goals_against"],
        home_stats["avg_shots"], home_stats["avg_possession"], h2h_stats["head_to_head"],
        h2h_stats["head_to_head_goals"], away_stats["opponent_encoded"]
    ]

    # Make prediction
    prediction = model.predict([features])[0]
    result_mapping = {1: "Win", 0: "Draw", -1: "Loss"}
    prediction_text = result_mapping.get(prediction, "Unknown")

    return jsonify({"home_team": home_team, "away_team": away_team, "prediction": prediction_text})


@app.route("/routes", methods=["GET"])
def show_routes():
    return jsonify([str(rule) for rule in app.url_map.iter_rules()])
    
# Run Flask app
if __name__ == "__main__":
    print("✅ Flask is starting...")
    # Debugging: Print all registered routes when Flask starts
    print("✅ Registered Routes:")
    for rule in app.url_map.iter_rules():
        print(rule)

    app.run(debug=True)


✅ Flask is starting...
✅ Registered Routes:
/static/<path:filename>
/predict
/routes
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\lebda\AppData\Local\Programs\Python\Python313\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
!jupyter nbconvert --to script app.ipynb

[NbConvertApp] Converting notebook app.ipynb to script
[NbConvertApp] Writing 2298 bytes to app.py


In [4]:
import os
print(os.listdir())  # Lists all files in the directory

['.ipynb_checkpoints', 'app.ipynb', 'app.py', 'app.py.py', 'data_prep.ipynb', 'head_to_head_stats.csv', 'match_predictor.ipynb', 'match_predictor.pkl', 'premier_league_2023_24.csv', 'test_flask.ipynb']
